# pgmpy

In [121]:
import numpy as np
import pandas as pd

# Gathering data

In [122]:
import quandl as qd
qd.ApiConfig.api_key = "_4PBEc5xxKVoQoYxWcgG"

In [123]:
#def get_from_quandl
        

# Creating network

In [174]:
from pgmpy.models import BayesianModel
import abc

In [175]:
class Basic:
    def __init__( self, params, start=0,lag=0):
        self.model = BayesianModel()
        for t in range(start,start+lag+1):
            for s in params:
                self.model.add_node(Basic.get(s,t))
    @staticmethod
    def get(param, lag):
        return param+'_'+str(lag)

In [184]:
class BNGrid(Basic):
    def __init__( self, params,depth,start=0,lag=0):
        if (depth>lag):
            raise
        super().__init__(params,start,lag)
        for t in range(start,start+lag+1):
            for s in params:
                x = BNFull.get(s,t)
                for tt in range(t+1,min(start+lag+1,t+depth+1)):
                    for ss in params:
                        par = BNFull.get(ss,tt)
                        self.model.add_edge(par,x)

In [185]:
class BNFull(BNGrid):
    def __init__( self, params,start=0,lag=0):
        super().__init__(params,depth=1,start=start,lag=lag)

In [186]:
class BNCascade(BNFull):
    def __init__(self,params,lag,target):
        super().__init__(params,start=1,lag=lag-1)
        target = BNCascade.get(target,0)
        self.model.add_node(target)
        for x in self.model.nodes():
            if (x!=target):
                self.model.add_edge(x,target)

In [187]:
network = BNCascade(params=['GOLD','USD'],lag=2,target='Amghezi')

In [188]:
network.model.edges()

[('GOLD_1', 'Amghezi_0'),
 ('USD_1', 'Amghezi_0'),
 ('GOLD_2', 'GOLD_1'),
 ('GOLD_2', 'USD_1'),
 ('GOLD_2', 'Amghezi_0'),
 ('USD_2', 'GOLD_1'),
 ('USD_2', 'USD_1'),
 ('USD_2', 'Amghezi_0')]

In [189]:
network = BNGrid(params=['GOLD','USD'],depth=2,lag=2)

In [190]:
network.model.edges()

[('GOLD_1', 'GOLD_0'),
 ('GOLD_1', 'USD_0'),
 ('USD_1', 'GOLD_0'),
 ('USD_1', 'USD_0'),
 ('GOLD_2', 'GOLD_0'),
 ('GOLD_2', 'USD_0'),
 ('GOLD_2', 'GOLD_1'),
 ('GOLD_2', 'USD_1'),
 ('USD_2', 'GOLD_0'),
 ('USD_2', 'USD_0'),
 ('USD_2', 'GOLD_1'),
 ('USD_2', 'USD_1')]